In [114]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
%matplotlib inline

countvec = CountVectorizer()

stop = stopwords.words('english')

#read file
items = pd.read_csv('/users/brianjenney/desktop/datascience/goiitems/items.csv',error_bad_lines=False, encoding='utf-8', engine='c')

#only get columns/rows we need
items = items.ix[1:,2:7]

#rename columns
items.columns=['ItemNum', 'Class', 'SubClass', 'Desc']

#keep array items name to append to wordbag list
descArray = items['Desc']

#strip stop words
items['Desc'].apply(lambda x: [item for item in x if item not in stop])

#strip numbers from desc
items['Desc'] = items['Desc'] .str.replace('\d+', '')

#create data frame with doc matrix
dm = pd.DataFrame(countvec.fit_transform(items['Desc']).toarray(), columns=countvec.get_feature_names())

#match the indexes
dm = dm.reset_index(drop=True)
items = items.reset_index(drop=True)

#concatenate the frames
wordbag = pd.concat([dm, items], axis=1)

#remove the description column and only have numeric data
wordbag = wordbag.iloc[:, :-1]

#create clusters from documents
estimator = KMeans(n_clusters=5)
estimator.fit(wordbag)

#add categories and descriptions to the dataframe
wordbag['Category'] = estimator.labels_
# wordbag['Desc'] = descArray

wordbag = wordbag.sort_values('Category', ascending=False)

wordbag

,alm,almond,arroz,assorted,bar,bars,beer,ben,berry,betty,...,variety,voluntary,watchers,weight,wild,wonder,ItemNum,Class,SubClass,Category
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,278282,36,93,4
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,342949,36,93,4
36,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,600632,36,93,3
28,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,567477,36,93,3
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,561672,36,93,3
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,561673,36,93,3
24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,561676,36,93,3
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,562589,36,93,3
26,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,563733,36,93,3
39,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,608925,36,93,3
